<p style="text-align: center">
<img src="./images/landsat_8_rend-sm1.png" width=250 alt="HoloViz Logo"></img>
</p>

# Preprocessing Data

---

## Overview
The last notebook ([Data Ingestion](01_Data_Ingestion)) demonstrated how to access the data. The next step is to ensure that the data has been appropriately prepared for whatever analysis we intend to run. As we will be directly comparing images of a specific lake across time points, it would be very useful if these images were on the same underlying coordinate grid. In this notebook, we will preprocess the data to reshape and align our images for efficient consumption by analysis steps.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necessary |  |
| [Intake](https://intake.readthedocs.io/en/latest/index.html) | Necessary |  |
| [Coordinate Reference Systems: EPSG codes](https://pygis.io/docs/d_understand_crs_codes.html#epsg-codes)| Helpful | |
| [Geoviews resampling grids](http://geoviews.org/user_guide/Resampling_Grids.html)| Helpful | |

- **Time to learn**: 20 minutes.

---

## Imports

In [ ]:
import intake
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import cartopy.crs as ccrs
import geoviews as gv
import hvplot.xarray
import warnings
warnings.simplefilter('ignore', FutureWarning)

## Loading data

If you haven't already, we highly recommended that you check out the previous tutorial for more information on loading data. Here, we'll jump right to using Intake to read in chunks of small versions of the Landsat 5 and Landsat 8 data.

In [ ]:
cat = intake.open_catalog('./data/catalog.yml')

In [ ]:
cat.landsat_8_small.container

We know that our Landsat data will ultimately be an xarray object. Let's load the metadata with Dask:

In [ ]:
landsat_5_da = cat.landsat_5_small.to_dask()
landsat_8_da = cat.landsat_8_small.to_dask()

In [ ]:
landsat_8_da

## Calculate the Vegetation Index

To motivate our preprocessing, let's conduct a simple analysis ot the Normalized Difference Vegetation Index [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) for each of these image sets as follows:

\begin{align}
{NDVI} & = \frac{NIR-Red}{NIR+Red}
\end{align}

where Red and NIR stand for the spectral reflectance measurements acquired in the Red (visible) and near-infrared regions, respectively. Note that in Landsat 5 the Red and NIR bands are stored in bands 3 and 4 respectively whereas in Landsat 8 the Red and NIR are bands 4 and 5.

In [ ]:
NDVI_1988 = (landsat_5_da.sel(band=4) - landsat_5_da.sel(band=3)) / (landsat_5_da.sel(band=4) + landsat_5_da.sel(band=3))

In [ ]:
NDVI_1988_plot = NDVI_1988.hvplot.image(x='x', y='y', geo=True, clim=(-1,1), title='NDVI 1988', rot=45, cmap='viridis')
NDVI_1988_plot

In [ ]:
NDVI_2017 = (landsat_8_da.sel(band=5) - landsat_8_da.sel(band=4)) / (landsat_8_da.sel(band=5) + landsat_8_da.sel(band=4))

In [ ]:
NDVI_2017_plot = NDVI_2017.hvplot.image(x='x', y='y', geo=True, clim=(-1,1), title='NDVI 2017', rot=45, cmap='viridis')
NDVI_2017_plot

We can calculate the difference between these two years by subtracting one from the other.

In [ ]:
NDVI_diff = NDVI_2017 - NDVI_1988
NDVI_diff_plot = NDVI_diff.hvplot.image(x='x', y='y', geo=True, cmap='coolwarm', clim=(-1,1), title='NDVI 2017 - 1988', rot=45)
NDVI_diff_plot

Notice how pixelated that image looks. What is going on here? To figure it out, let's take a look at the shape of `diff`.

In [ ]:
NDVI_diff.shape

That's a lot smaller than our NDVI for each year. What is happening is that when we compute the difference on the data we only get values where there are values for each year in the same grid cell. Since the cells are on a different resolution this only happens once every so often. What we'd rather do is interpolate to the same grid and then do our computations on that.


## Combine data from overlapping grids

These two sets of Landsat bands cover roughly the same area but were taken in 1988 and 2017. We have modified them to have different resolutions, different numbers of grid cells per image, and different x and y offsets. We can see that by printing the first `x` value for each year and seeing that they are not equivalent.

In [ ]:
print(NDVI_1988.x[0].values)
print(NDVI_2017.x[0].values)

We can also do a quick check of resolution by subtracting the second x value from the first x value for each year.

In [ ]:
print((NDVI_1988.x[1] - NDVI_1988.x[0]).values)
print((NDVI_2017.x[1] - NDVI_2017.x[0]).values)

### Select region of interest

We'll define a central point in lat, lon convert that point to the *coordinate reference system (CRS)* of the data, and then use the area around the central point as the Region of Interest (ROI). 

The first step is getting the CRS. This information is stored in the attributes of our original landsat data. Let's take a look at it:

In [ ]:
landsat_8_da.crs

This CRS is referenced by an EPSG code. We can see more about this specific code at [EPSG.io](https://epsg.io/32611). Read more about EPSG codes in this [Coordinate Reference Systems: EPSG codes](https://pygis.io/docs/d_understand_crs_codes.html#epsg-codes) online book chapter. Generally, the `+init=<authority>:<code>` syntax [is being deprecated](https://pyproj4.github.io/pyproj/stable/gotchas.html#init-auth-auth-code-should-be-replaced-with-auth-auth-code), and `<authority>:<code>` is the preferred initialization method.

We can and convert this CRS into a `cartopy.crs` object using the `cartopy.crs.epsg` method.

In [ ]:
crs = ccrs.epsg(32611)

In [ ]:
# TODO: create issue about geoviews' proj_to_cartopy not working and process_crs needing to substring 'epsg:32611'

In [ ]:
crs.__repr__()

In [ ]:
crs.area_of_use

In [ ]:
crs_box = [(crs.x_limits[0], crs.y_limits[0]), (crs.x_limits[0], crs.y_limits[1]),
           (crs.x_limits[1], crs.y_limits[1]), (crs.x_limits[1], crs.y_limits[0])]

crs_extent = gv.Polygons(crs_box, crs=crs).options(alpha=0.4, line_alpha=0)

(gv.feature.coastline * crs_extent).options(title='EPSG:32611', global_extent=True)

If we know already know our center point in terms of a latitude, longitude point (from a `cartopy.crs.PlateCarree()` projection), we can transform this into the `crs` of our data using the `transform_point` method:

In [ ]:
x_center, y_center = crs.transform_point(-118.7081, 38.6942, ccrs.PlateCarree())

In [ ]:
y_center

But what if we didn't happen to know the center point in terms of latitude and longitude? Since we are using interactive plots with `hvplot`, we can look at one of our previous plots of the lake and hover over the region that we want to use as a center point to reveal the coordinates in latitude, longitude (created by plotting with `geo=True`). 

Alternatively, we could just hover over a plot of our data that is still in our data `crs` projection (default is `geo=False`) to directly get our `x_center` and `y_center`. Try it below - hover over the center of the lake to reveal the x and y points:

In [ ]:
NDVI_2017.hvplot.image(x='x', y='y', width=400, clim=(-1,1), title='NDVI 2017', rot=45, cmap='viridis')

By hovering, our estimated center point reveals itself. Don't worry about this being an estimate of the center point, our focus here is to get a region that captures the lake, so a rough estimate is sufficient.

In [ ]:
x_center = 3.515e+5
y_center = 4.284e+6

Now we just need to define the area that we are interested in around this point. In this case we'll use a 30 km box around the center point.

In [ ]:
buffer = 1.5e4

xmin = x_center - buffer
xmax = x_center + buffer
ymin = y_center - buffer
ymax = y_center + buffer

bounding_box = [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]

Let's just check that the bounding box captures the lake:

In [ ]:
roi = gv.Polygons(bounding_box, crs=crs)

In [ ]:
gv.tile_sources.EsriImagery * roi.options(alpha=0.3)

### Regrid

We can use this region to define a new grid onto which we will interpolate our data. We will set the resolution of our new grid within the bounding box to roughly match the resolution of our original images.

In [ ]:
res = 200
x = np.arange(xmin, xmax, res)
y = np.arange(ymin, ymax, res)
x.shape

We will use linear interpolation to calculate the values for each grid cell.

In [ ]:
NDVI_2017_regridded = NDVI_2017.interp(x=x, y=y)
NDVI_1988_regridded = NDVI_1988.interp(x=x, y=y)

Let's compare our original data to this regridded form.

In [ ]:
NDVI_2017_regridded_plot = NDVI_2017_regridded.hvplot.image(x='x', y='y', clim=(-1,1), title='NDVI 2017 regridded', geo=True, rot=45, cmap='viridis')
NDVI_1988_regridded_plot = NDVI_1988_regridded.hvplot.image(x='x', y='y', clim=(-1,1), title='NDVI 1988 regridded', geo=True, rot=45, cmap='viridis')

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Note</p>
    With hvPlot, you can render multiple plots using the `+` operator, and stack them into rows by defining the number of columns with `.cols`. When possible, hvPlot will automatically link the plots so you can zoom and pan on one image and keep them in sync.
</div>


Zoom into one of the images below to see the result of interpolating on this new common grid. The regridded plots (bottom row) will have a consistent pixelation compared to the original NDVI data (top row).

In [ ]:
(NDVI_1988_plot + NDVI_2017_plot + NDVI_1988_regridded_plot + NDVI_2017_regridded_plot).cols(2)

### Combining the data 


Now that we have our data on the same grid we can combine our two years into one `xarray` object. We will treat the years as names and create an `xarray.Dataset` - a group of named `xarray.DataArray`s that share some of the same coordinates.

In [ ]:
ds_regridded = xr.Dataset({'NDVI_1988': NDVI_1988_regridded, 'NDVI_2017': NDVI_2017_regridded})
ds_regridded

### Vizualizing output


We can now reference each year from the same object, and plot the arrays side by side:

In [ ]:
ds_regridded.NDVI_1988.hvplot.image(x='x', y='y', geo=True, clim=(-1, 1), title='NDVI 1988', cmap='viridis')  +\
ds_regridded.NDVI_2017.hvplot.image(x='x', y='y', geo=True, clim=(-1, 1), title='NDVI 2017', cmap='viridis')

Or we can calculate and plot the difference between the two years:

In [ ]:
diff_regridded = ds_regridded['NDVI_2017'] - ds_regridded['NDVI_1988']
diff_regridded

In [ ]:
NDVI_diff_regridded_plot = diff_regridded.hvplot.image(x='x', y='y', geo=True, rot=45, cmap='coolwarm', clim=(-1,1), title='NDVI 2017 - 1988')
NDVI_diff_regridded_plot

As the Vegetation Index, will generally give a lower value where water is present, you can clearly see a large positive change along the edge of the lake indicating a reduction in the size of the lake over this time period.

In [ ]:
# TODO: investigate Normalized difference water index (NDWI) in place, or in addition to, NVDI.

## Side-note: Resampling


Depending on your analysis, it may be beneficial to downsample your data. Especially early on during analysis development, running computations on full resolution data may eat into valuable time that you could otherwise use iterating, debugging, and improving your workflow. Luckily, downsampling xarray data is made pretty easy by grouping the values into bins based on the desired resolution and taking some aggregation (like the mean) on each of those bins.

In [ ]:
res_1000 = 1e3
x_1000 = np.arange(xmin, xmax, res_1000)
y_1000 = np.arange(ymin, ymax, res_1000)

We'll use the left edge as the label for now.

In [ ]:
diff_res_1000 = (diff_regridded
    .groupby_bins('x', x_1000, labels=x_1000[:-1]).mean(dim='x')
    .groupby_bins('y', y_1000, labels=y_1000[:-1]).mean(dim='y')
    .rename(x_bins='x', y_bins='y')
)
diff_res_1000

In [ ]:
diff_res_1000.hvplot.image(x='x', y='y', geo=True, rot=45, cmap='coolwarm', clim=(-1,1), title='Downsampled')

And there you go, nicely downsampled data ready for some fast computations. Note, we have regridded the data while treating the space as flat, which is fine for our demonstration purposes at a small-ish scale. However, despite what you may have heard on the interweb, the earth is not flat, and treating it as such can give less accurate results when working with a spherical space. For more information about spherical resampling methods, check out the [GeoViews resampling grids](http://geoviews.org/user_guide/Resampling_Grids.html) page, as it describes using different grid types including rectilinear, curvilinear grids and trimeshes.

---

## Summary
With the target of preparing our data for further analysis, we've learned how to ensure that our data are on a consistent coordinate grid. Again, this is especially important if we are performing operations across the datasets, like taking the difference between their images. Although preprocessing can sometimes be tedious, it's a very critical part of the workflow, and often only needs to be done once (or a small number of times) before being able to apply the processed data to a large number of different analyses.

### What's next?
Now that we know how to prepare data, it's time to proceed to analysis, where we will explore a some simple machine learning approaches.


## Resources and references
- Authored/adapted by Demetris Roumis circa Dec, 2022
- This cookbook was inspired by the [EarthML](https://github.com/pyviz-topics/EarthML) tutorial. See a list of the EarthML contributors [here](https://github.com/pyviz-topics/EarthML/graphs/contributors).
<a href="https://github.com/pyviz-topics/EarthML/graphs/contributors">
  <img src="https://contrib.rocks/image?repo=pyviz-topics/EarthML" />
</a>
- The landsat 8 banner image is from [NASA](https://svs.gsfc.nasa.gov/10812)